In [2]:
# install konlpy_ kkma
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 33.7 MB/s eta 0:00:00


In [3]:
# 2

import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/NLP labs/Week7_1_IntroToNLP_ratings_mini.txt", sep="\t", encoding='utf8')
# print(df) # id, document, label(0:negative, 1: positive)

print('negative reviews : ',len(df[df['label']==0])) # df[df['label'==0].count]
print('positive reviews : ',len(df[df['label']==1]))


negative reviews :  2483
positive reviews :  2517


In [4]:
# 3-1
negative_df = list(df[df['label']==0]['document']) # 뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
positive_df = list(df[df['label']==1]['document'])
# print(negative_df)


# 각 리뷰 형태소 분석

from konlpy.tag import Kkma
kkma = Kkma()

# morphs로 형태소 분석 =====================================
negative_df_morphs = []
positive_df_morphs = []

for i in negative_df:
  negative_df_morphs.append(kkma.morphs(str(i))) # 부정리뷰 형태소 분석
  # print(negative_df_morphs) # 값 체크

for i in positive_df:
  positive_df_morphs.append(kkma.morphs(str(i))) # 긍정리뷰 형태소 분석


# nouns로 형태소 분석 ========================================
negative_df_nouns = []
positive_df_nouns = []

for i in negative_df:
  negative_df_nouns.append(kkma.nouns(str(i))) # 부정리뷰 형태소 분석
  # print(negative_df_nouns) # 값 체크


for i in positive_df:
  positive_df_nouns.append(kkma.nouns(str(i))) # 긍정리뷰 형태소 분석


In [5]:
# 3-2 : 각각 분류된 리뷰에 들어간 형태소들의 출현 빈도를 count

# print(negative_df_morphs) #[['뭐', '야', '이', '평점', '들', '은', '....', '나쁘', '지', '는', '않', '지만', '10', '점', '짜리', '는', '더더욱', '아니', '잖아'] ...]
negative_df_morphs_list = [y for x in negative_df_morphs for y in x]
positive_df_morphs_list = [y for x in positive_df_morphs for y in x]
negative_df_nouns_list = [y for x in negative_df_nouns for y in x]
positive_df_nouns_list = [y for x in positive_df_nouns for y in x]


from collections import Counter
n_morph_cnt = dict(Counter(negative_df_morphs_list)) # kkma.morph 으로 분류된 문장의 형태소 출현빈도
p_morph_cnt = dict(Counter(positive_df_morphs_list))
n_noun_cnt = dict(Counter(negative_df_nouns_list)) # kkma.noun 으로 분류된 문장의 형태소 출현빈도
p_noun_cnt = dict(Counter(positive_df_nouns_list))

# 결과값 출력
print(n_morph_cnt) #{'이': 1923, '하': 1474, 'ㄴ': 1251, '는': 1221, ...}
print(p_morph_cnt)
print(n_noun_cnt) #{'영화': 776, '점': 194, '평점': 133, '1': 124, ...}
print(p_noun_cnt)

{'GDNTOPCLASSINTHECLUB': 1, '뭐': 107, '야': 73, '이': 1923, '평점': 137, '들': 331, '은': 543, '....': 91, '나쁘': 18, '지': 357, '는': 1221, '않': 126, '지만': 108, '10': 44, '점': 213, '짜리': 13, '더더욱': 1, '아니': 159, '잖아': 8, '지루': 108, '하': 1474, '은데': 23, '완전': 43, '막장': 23, '임': 42, '...': 447, '돈': 68, '주고': 17, '보': 600, '기에': 15, '3': 58, 'D': 5, '만': 223, '었': 532, '어도': 51, '별': 63, '다섯': 1, '개': 64, '주': 151, '을': 512, '터': 19, 'ㄴ데': 79, '..': 555, '왜': 112, '로': 183, '나오': 132, '아서': 50, '저': 52, '의': 390, '심기': 1, '를': 292, '불편': 12, '게': 275, '죠': 12, '??': 45, '진정': 4, 'ㄴ': 1251, '쓰레기': 84, '마치': 2, '미국': 15, '애': 49, '니': 104, '에': 515, '서': 70, '튀어나오': 2, '듯하': 15, '창의력': 1, '없': 355, '로봇': 3, '디자인': 1, '부터': 34, '가': 540, '아': 380, ',': 297, '고개': 1, '젖': 2, 'ㄴ다': 156, '갈수록': 14, '개판되가': 1, '중국': 11, '영화': 973, '유치': 50, '고': 751, '내용': 96, '음': 154, '폼잡': 1, '다': 985, '끝나': 25, 'ㅁ': 205, '말': 104, '도': 667, '안되': 40, '무기': 1, 'cg': 9, '남무': 1, '그립': 3, '동사': 1, '서독': 1, '같': 138, '

In [15]:
# 4-1 : 긍정단어와 부정단어 추출 - morph 이용

'''
단어 조건 :
1. 해당 단어가 전체 리뷰중 10번 이상 출현해야 함
해당 단어의 출현 빈도가 하나의 label에 90% 이상 치우쳐 있어야 함
'''

neg_words = []
pos_words = []

for neg in negative_df_morphs_list:
  for pos in positive_df_morphs_list:
    if neg == pos:

      if n_morph_cnt[neg] + p_morph_cnt[neg] < 10 : # 전체 출현 빈도 < 10 : 계산 필요X
        break
      else: # 전체 출현 빈도 >= 10 : 긍/부정 비율 계산
        sent_rate = n_morph_cnt[neg] / (n_morph_cnt[neg] + p_morph_cnt[neg])

        if sent_rate >= 0.9 : # 부정의 경우
          neg_words.append(neg)
          break

        elif 1-sent_rate <=0.1: # 긍정의 경우
          pos_words.append(neg)
          break

for pos in positive_df_morphs_list:
  for neg in negative_df_morphs_list:
    if neg == pos:
      if n_morph_cnt[neg] + p_morph_cnt[neg] < 10 : # 전체 출현 빈도 < 10 : 계산 필요X
        break
      else: # 전체 출현 빈도 >= 10 : 긍/부정 비율 계산
        sent_rate = p_morph_cnt[pos] / (n_morph_cnt[pos] + p_morph_cnt[pos])

        if sent_rate >= 0.9 : # 긍정의 경우
          pos_words.append(neg)
          break

        elif 1-sent_rate <=0.1: # 부정의 경우
          neg_words.append(neg)
          break

neg_words_morph = list(set(neg_words))
pos_words_morph = list(set(pos_words))


In [16]:
# kkma.morph 로 분류한 경우 긍/부정 단어
print('Negative words : ',neg_words_morph)
print('Positive words : ',pos_words_morph)

Negative words :  ['아까', '접', '성우', '뭐하', 'OOO', '재미없', '장난', '아류', '어이', '수준', '어설프', '그만', '로써', '유치', '접하', '역겹', '차라리', '짜증나', '너', '쓰레기', '예산', '0', '엉성', '아깝', '설정', '노출', '그나마', '너무하', '망', 'ㅡㅡ', '떨어지', '언', '삼류', '낭비', '짜증', '이딴']
Positive words :  ['감동적', '꽤', '최고', '누나', '흐르', '낮', '걸작', '잔잔', '환', '감탄', '^^', '유쾌', 'good', '소름', '즐겁', '담기', '춤', '짱', '벌써', '굿', '생각나', '대박', '잊', '새롭', '오빠', '삶', '깊', '행복']


In [11]:
# 4-2 : 긍정단어와 부정단어 추출 - noun 이용

'''
단어 조건 :
1. 해당 단어가 전체 리뷰중 10번 이상 출현해야 함
해당 단어의 출현 빈도가 하나의 label에 90% 이상 치우쳐 있어야 함
'''

neg_words = []
pos_words = []

for neg in negative_df_nouns_list:
  for pos in positive_df_nouns_list:
    if neg == pos:

      if n_noun_cnt[neg] + p_noun_cnt[neg] < 10 : # 전체 출현 빈도 < 10 : 계산 필요X
        break
      else: # 전체 출현 빈도 >= 10 : 긍/부정 비율 계산
        sent_rate = n_noun_cnt[neg] / (n_noun_cnt[neg] + p_noun_cnt[neg])

        if sent_rate >= 0.9 : # 부정의 경우
          neg_words.append(neg)
          break

        elif 1-sent_rate <=0.1: # 긍정의 경우
          pos_words.append(neg)
          break

for pos in positive_df_nouns_list:
  for neg in negative_df_nouns_list:
    if neg == pos:
      if n_noun_cnt[neg] + p_noun_cnt[neg] < 10 : # 전체 출현 빈도 < 10 : 계산 필요X
        break
      else: # 전체 출현 빈도 >= 10 : 긍/부정 비율 계산
        sent_rate = p_noun_cnt[pos] / (n_noun_cnt[pos] + p_noun_cnt[pos])

        if sent_rate >= 0.9 : # 긍정의 경우
          pos_words.append(neg)
          break

        elif 1-sent_rate <=0.1: # 부정의 경우
          neg_words.append(neg)
          break


neg_words_noun= list(set(neg_words))
pos_words_noun = list(set(pos_words))


In [17]:
# kkma.noun으로 분류한 경우 긍/부정 단어
print('Negative words : ',neg_words_noun)
print('Positive words : ',pos_words_noun)

Negative words :  ['쓰레기영화', '성우', '장난', '어이', '수준', '3점', '발연기', '유치', '너', '쓰레기', '예산', '질', '0', '설정', '노출', '망', '삼류', '낭비', '짜증']
Positive words :  ['감동적', '굿', '감탄', '만점', '최고', '삶', '소름', '넘', '대박', '걸작', '춤', '환', '짱']


In [21]:
# 6-1 : 긍/부정 morph
# 긍정 단어와 부정단어가 많이 나온 문장 추출

def count_words(review, words):
    return sum(review.count(word) for word in words)

# 긍정 리뷰 정렬
sorted_positive_reviews_m = sorted(
    positive_df,
    key=lambda review: count_words(review, pos_words_morph),
    reverse=True
)

# 부정 리뷰 정렬
sorted_negative_reviews_m = sorted(
    negative_df,
    key=lambda review: count_words(review, neg_words_morph),
    reverse=True
)

# 결과 출력, 내림차순 10문장
print("Positive reviews_using morph :")
for review in sorted_positive_reviews_m[:10]:
    print(review)

print()
print("Negative reviews_using morph:")
for review in sorted_negative_reviews_m[:10]:
    print(review)



Positive reviews_using morph :
배우 이민호와 김희선의 어울리지 않을것 같은 어울림이 묘한 매력으로 다가오는 신의 ^^ 타임스립드라마의 최고봉이라 할 수 있답니다. 잔잔하게 애잔한 신의의 매력으로 풍덩 빠져보심이 어떨런지 ^^ 내인생 최고의 드라마였습니다. 적극 추천합니다.
우리에게 행복을 가져다 주는 이들보다 끔찍한 이들이 더 많은 게 현실이다. 우리를 행복하게 해주는 사람들과 함께 하는 것도 중요하지만 그보다 더 중요한 것은 나 스스로 행복해져야 한다는 것이다. 스스로 행복한 사람이 타인도 행복하게 만들 수 있다.
완전 재미있어요~ 절대 놓치지 않고 봐요...무슨일이 있어도...최고의 재미예요^^글구 모든 배우들이 연기 정말 잘하시네요....최고^^만점의 점수^^
가슴 깊이 울림을 주는 감동적인 영화다.마지막 자막이 올라 갈때까지 흐르는 눈물...이런 아름다운 영화를 이제야 감상하다니...케빈 코스트너,클린트 이스트우드 감독님...존경합니다~~^^
최고 최고 최고 최고 한번만더틀어주삼!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
액션조 짱이고 여운도 짱이고, 캐스팅도 짱 스토리도 짱!정말 잼있게 본 들마였어요><!!
아직까지 잊지 못하는 영화. 제목부터 마음에 잔잔하게 와 닿는다. 영화 장면중에 소낙비를 피해놀라 달려 가는 부분은 정말 ☆어느 멋진날~☆ 햇살 비춰진 행복한 빗속을 달리듯 느껴졌었다
세분 비구니들이 꼭 자기자신을 찾기를 간절히 바랍니다.약간의 눈물은 흘렸지만 잔잔한 그들의 모습에서 나의 삶을 뒤돌아본 너무나좋은 느낌의 다큐였습니다. 진심으로 이런 분들의 삶을 으원하며 감독님의 열정에 감사 드립니다.
영국에도 팍팍한 탄광노동자의 삶이 있구나. 자신의 재능을 발휘할 수 있는 삶을 찾아가는 소년은 행복하다.
사람들이 일상을 대할땐 언제나 신경적이고 지루하하는 경향이 심하다. 삶이란 생각보다 길고 어려운 난간에 봉착하기 마련이기때문인것같다그렇게일상에지내다보면소중한감성을잊고살아오는것같다 이영화를 보고

In [22]:
# 6-2 : 긍/부정 noun
# 긍정 단어와 부정단어가 많이 나온 문장 추출

def count_words(review, words):
    return sum(review.count(word) for word in words)

# 긍정 리뷰 정렬
sorted_positive_reviews_n = sorted(
    positive_df,
    key=lambda review: count_words(review, pos_words_noun),
    reverse=True
)

# 부정 리뷰 정렬
sorted_negative_reviews_n = sorted(
    negative_df,
    key=lambda review: count_words(review, neg_words_noun),
    reverse=True
)

# 결과 출력, 내림차순 10문장
print("Positive reviews_using noun :")
for review in sorted_positive_reviews_n[:10]:
    print(review)

print()
print("Negative reviews_using noun:")
for review in sorted_negative_reviews_n[:10]:
    print(review)



Positive reviews_using noun :
최고 최고 최고 최고 한번만더틀어주삼!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
2000년대 영화 중 손에 꼽을 정도로 걸작이라 생각한다. 전신마비 된 주인공의 내면을 환상과 꿈을 넘나들며, 대단한 비유와 은유가 담긴 연출이 돋보이고 한편의 다큐를 보는듯한 촬영도 인상적이며 400번의 구타를 오마쥬한 트래킹 쇼트도 최고였다.
액션조 짱이고 여운도 짱이고, 캐스팅도 짱 스토리도 짱!정말 잼있게 본 들마였어요><!!
최고진짜대박겨울만대면보고진짜돌려보기안좋아하는데도열번도넘게본드라마
완전 재미있어요~ 절대 놓치지 않고 봐요...무슨일이 있어도...최고의 재미예요^^글구 모든 배우들이 연기 정말 잘하시네요....최고^^만점의 점수^^
류덕환씨 완전 팬~!!세계영화를뛰어넘는 드라마중 최고의 드라마!!
정말 최고다..춤출때마다 소름돋네
갠적으로 짱구 영화중 젤잼잇엇음 이건 봐도봐도 재밌는 영화 완전강추!!!!bb짱구 영화중 짱 다미도 이쁘고 ㅎㅎ
최고다 최고 최고의 액션영화 눈을 뗄수업는 긴장감과 액션을 보여준다
대박 재밋네요 쌤해밀턴 광고로 대박나길 넘 귀엽다 ㅋㅋㅋ쌤 화이팅

Negative reviews_using noun:
잼없다.. 결말도 그지같고, 지붕킥이 더 10000000000000배 낫다
0000000000
한국영화의 암흑기인 1980년대에 흔히 볼수 있는 전형적인 영화입니다. 이영화만 봐도 왜 80년대가 한국영화의 암흑기인지 잘 알수 있죠..20세기 한국영화 수준은 60년대>90년대>70년대>50년대>>>>>>80년대 ㅎㅎ
cg수준: 전투기가 나타났다 사라지는 기적/ 음향수준:삐뽀삐뽀 콰쾅콰쾅 빵빵빠앙 x 100/ 카메라 수준: 치직치칙깜빡깜빡x1000/자매수준:카메라꺼!켜!꺼!켜!
헐이다..5분이면 될 내용을 90분으로 늘인다고 애 썼수..이건뭐 변태 싸이코 패스 수준에 더럽게 연기 못하는 여배우 하나 벗긴거 말고는 봐줄 가치도 없음..차라리 애로를 보고 말지 쯧 별점은